# Contenido multimedia

# C2_S5 · Contenido multimedia (Demo profesor)
Meta: detectar medios en HTML, resolver URLs absolutas, elegir variantes de `srcset`, clasificar por tipo sin descargar.



In [3]:
import re, mimetypes
from urllib.parse import urljoin, urlparse, urlsplit, urlunsplit
from collections import Counter

import requests
from bs4 import BeautifulSoup


# ¿Dónde se declaran los medios?

🔹 1. Imágenes (<img>, <picture>/<source>)

👉 Link: https://www.w3schools.com/html/html_images.asp

Abre y busca en el código:


![C](declarandoImg.PNG)

Esa etiqueta img tiene un atributo src con la URL de la imagen.

👉 Ejemplo con picture/srcset:
https://developer.mozilla.org/en-US/docs/Web/HTML/Element/picture

En el código hay algo así:

![c](declarandoImg2.png)

🔹 2. Video (< video >/< source >)

👉 Link: https://www.w3schools.com/html/html5_video.asp

En el código verás algo así:

![w](declaracionVideo.png)

🔹 3. Audio (<audio>/<source>)

👉 Link: https://www.w3schools.com/html/tryit.asp?filename=tryhtml5_audio_all

Código:

![s](declaracionAudio.png)

🔹 4. Embebidos ( <    IFRAME  >   )

👉 Link: https://www.w3schools.com/html/html_iframe.asp

Ejemplo en el código:

Aquí el atributo src apunta a otra página embebida.

![a](iframe.png)

---


# Variantes en srcset


🔹 1. Ejemplo sencillo en Python: elegir la “mejor variante”

Supongamos que tenemos un srcset de una imagen, que trae varias resoluciones:

In [ ]:
srcset = {
    "320w": "https://sitio.com/imagen-320.jpg",
    "640w": "https://sitio.com/imagen-640.jpg",
    "1280w": "https://sitio.com/imagen-1280.jpg"
}

# Nuestro "objetivo" es 800px
objetivo = 800

# Regla light → usar la mayor variante que no se pase de 800px
mejor_light = max(
    [(int(k.replace("w", "")), v) for k, v in srcset.items() if int(k.replace("w", "")) <= objetivo], #Creo una lista de pares (ancho, url) solo con los anchos menores o iguales al objetivo.
    key=lambda x: x[0] # Le digo a max() que compare usando el primer elemento de la tupla (el ancho).
)

# Regla heavy → usar la mayor variante siempre, sin importar el objetivo
mejor_heavy = max(
    [(int(k.replace("w", "")), v) for k, v in srcset.items()], # Creo una lista de pares (ancho, url) con todas las opciones de imágenes, sin descartar ninguna.
    key=lambda x: x[0]
)

print("👉 Regla light:", mejor_light)
print("👉 Regla heavy:", mejor_heavy)


👉 Regla light: (640, 'https://sitio.com/imagen-640.jpg')
👉 Regla heavy: (1280, 'https://sitio.com/imagen-1280.jpg')


# Guardar


📘 Resumen por secciones
1) Guardar una imagen (binario simple)

Usamos r.content porque es un archivo binario (no texto).

Guardamos con wb → write binary.

👉 Ejemplo:

In [13]:
import requests

url = "https://www.w3.org/People/mimasa/test/imgformat/img/w3c_home.jpg"
r = requests.get(url, timeout=10)
r.raise_for_status()

with open("imagen_w3c.jpg", "wb") as f:
    f.write(r.content)

print("✅ Imagen guardada")


✅ Imagen guardada


2) Guardar un MP3

Igual que la imagen, es binario.

Sólo cambia la extensión.

👉 Ejemplo:

In [14]:
url = "https://www.soundhelix.com/examples/mp3/SoundHelix-Song-1.mp3"

r = requests.get(url, timeout=15)
r.raise_for_status()

with open("ejemplo_song.mp3", "wb") as f:
    f.write(r.content)

print("✅ MP3 guardado")


✅ MP3 guardado


3) Guardar un Video (streaming)

Como los videos pesan mucho, usamos stream=True + iter_content para leerlo por partes (chunks).

Así no llenamos la memoria RAM.

👉 Ejemplo:

In [19]:
import requests

url = "https://file-examples.com/storage/fe26d8c2d5f7e194c16f1a3/2017/04/file_example_MP4_480_1_5MG.mp4"

# streaming=True permite descargar en chunks
resp = requests.get(url, stream=True)

with open("ejemplo.mp4", "wb") as f:
    for chunk in resp.iter_content(chunk_size=1024):  # 1 KB por chunk (puedes usar 4096 o 8192)
        if chunk:  # filtra keep-alive chunks vacíos
            f.write(chunk)

print("Video guardado correctamente como ejemplo.mp4")



Video guardado correctamente como ejemplo.mp4


#### Calsaificar

![sss](Clasifica.png)

### MIME



1. Qué es MIME

MIME type = “Multipurpose Internet Mail Extensions type”.

Es un estándar para nombrar qué tipo de archivo es un recurso, en formato:

👉 Content-Type es un header HTTP que contiene el MIME.
Ejemplo: un servidor puede responder:

Content-Type: image/png


### ¿De qué me sirve clasificar?



Clasificar antes de descargar sirve para:

Priorizar: Si quieres solo imágenes, descartas audios/videos sin bajarlos.

Filtrar basura: Evitas guardar PDFs, HTML, etc. cuando buscabas medios multimedia.

Optimizar: Ahorras ancho de banda y CPU porque decides qué descargar de verdad.

Organizar: Puedes separar los recursos en carpetas (imgs/, videos/, audios/).

Ejemplo real:

Si haces un scraper y una página tiene 100 links, con classify() puedes ignorar los que apuntan a .pdf o .html, y quedarte solo con imágenes o audios.

In [ ]:
def classify(url: str, content_type: str | None) -> str:
    # Normalizamos los valores a minúscula para evitar problemas de mayúsculas
    ct = (content_type or "").lower()
    u = (url or "").lower()

    # 1. Si el servidor nos dio un Content-Type y dice "image/" → lo clasificamos como imagen.
    #    Si no, también revisamos por extensión .png, .jpg, etc.
    if "image/" in ct or u.endswith((".png",".jpg",".jpeg",".gif",".webp",".svg")):
        return "img"

    # 2. Lo mismo para video: preferimos Content-Type "video/"
    #    pero si no hay, revisamos extensiones conocidas.
    if "video/" in ct or u.endswith((".mp4",".webm",".ogg",".mov",".m4v")):
        return "video"

    # 3. Lo mismo para audio.
    if "audio/" in ct or u.endswith((".mp3",".ogg",".wav",".m4a")):
        return "audio"

    # 4. Detectamos enlaces que no apuntan a un archivo en sí,
    #    sino a un "embed" (ej: YouTube, Spotify, SoundCloud).
    if any(k in u for k in ("youtube.com","youtu.be","spotify.com","soundcloud.com")):
        return "embed"

    # 5. Si no encaja en ninguna regla, lo clasificamos como "other".
    return "other"



Cómo probarla individualmente 👇

Esto significa que ya sabes qué es cada link sin descargar nada.

In [34]:
# Lista de pruebas con URL + content-type
pruebas = [
    ("https://example.com/foto.png", "image/png"),
    ("https://example.com/video.mp4", "video/mp4"),
    ("https://example.com/cancion.mp3", "audio/mpeg"),
    ("https://youtube.com/watch?v=abc", None),
    ("https://example.com/documento.pdf", "application/pdf"),
]

# Probar la función con cada caso
for url, ct in pruebas:
    print(url, "->", classify(url, ct))


https://example.com/foto.png -> img
https://example.com/video.mp4 -> video
https://example.com/cancion.mp3 -> audio
https://youtube.com/watch?v=abc -> embed
https://example.com/documento.pdf -> other
